In [2]:
import os
from pathlib import Path
import sys

import arcpy
from arcgis.features import GeoAccessor
import pandas as pd

from ba_tools import utils

In [3]:
data = Path(os.path.abspath('../data'))
interim_dir = data/'interim'
raw_dir = data/'raw'
raw_gdb = raw_dir/'raw.gdb'

master_csv = interim_dir/'master_data.csv'
inrix_trips = raw_dir/'cust_count_inrix.csv'

In [15]:
master_df = pd.read_csv(master_csv).rename(columns={'ID': 'origin_id'})
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2476 entries, 0 to 2475
Columns: 7318 entries, origin_id to proximity_competition_kilometers_06
dtypes: float64(7315), int64(1), object(2)
memory usage: 138.2+ MB


In [16]:
master_df.head()

,origin_id,F1yearincrements_AGE0_CY,F1yearincrements_AGE1_CY,F1yearincrements_AGE2_CY,F1yearincrements_AGE3_CY,F1yearincrements_AGE4_CY,F1yearincrements_AGE5_CY,F1yearincrements_AGE6_CY,F1yearincrements_AGE7_CY,F1yearincrements_AGE8_CY,...,proximity_competition_kilometers_03,destination_competition_id_04,proximity_competition_traveltime_04,proximity_competition_kilometers_04,destination_competition_id_05,proximity_competition_traveltime_05,proximity_competition_kilometers_05,destination_competition_id_06,proximity_competition_traveltime_06,proximity_competition_kilometers_06
0,530530701003,31.0,25.0,30.0,28.0,35.0,35.0,32.0,24.0,37.0,...,25.960217,403218374.0,30.669763,26.656119,948351200.0,33.274677,31.095850,2890986.0,41.531836,34.025060
1,530530714071,27.0,27.0,27.0,25.0,25.0,25.0,25.0,25.0,25.0,...,6.134528,805003878.0,12.530292,8.456627,401706143.0,14.085878,9.183802,460555345.0,12.203891,9.231713
2,530530714072,22.0,23.0,15.0,18.0,22.0,11.0,17.0,29.0,24.0,...,5.910509,460555345.0,9.562275,6.501900,805003878.0,13.746727,9.084260,667567382.0,12.891244,10.126950
3,530530714073,22.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,22.0,...,6.162934,460555345.0,13.333669,7.594656,667567382.0,15.409227,10.195625,667588727.0,17.839612,11.169613
4,530530714112,23.0,22.0,22.0,23.0,22.0,22.0,22.0,22.0,21.0,...,6.268874,718164872.0,10.085421,6.620393,667588727.0,11.028254,7.036569,996721817.0,11.014189,7.203595


In [25]:
competitor_location_layer = str(raw_gdb/'sea_ace_comp')
competitor_id_field = 'LOCNUM'
competitor_brand_name_field = 'CONAME'

comp_lst = [r for r in arcpy.da.SearchCursor(competitor_location_layer, [competitor_id_field, competitor_brand_name_field])]
comp_df = pd.DataFrame(comp_lst, columns=['competition_destination_id', 'competition_brand_name'])
comp_df.head()

,competition_destination_id,competition_brand_name
0,460555345,CAPITOL LUMBER & DOOR
1,502088164,BUILDERS FIRST SOURCE
2,718164872,RANDLES SURE GROW TOPSOIL
3,805003878,NORTHERN ACOUSTICAL SYSTS CO
4,216099309,LOWE'S HOME IMPROVEMENT


In [26]:
comp_cat_df = utils.add_store_name_category(comp_df, 'competition_brand_name', location_count_threshold=1)
comp_cat_df.rename(columns={'dest_name_category': 'competition_name_category'}, inplace=True)
comp_cat_df.head()

,competition_destination_id,competition_brand_name,competition_name_category
0,460555345,CAPITOL LUMBER & DOOR,INDEPENDENT
1,502088164,BUILDERS FIRST SOURCE,BUILDERS FIRST SOURCE
2,718164872,RANDLES SURE GROW TOPSOIL,INDEPENDENT
3,805003878,NORTHERN ACOUSTICAL SYSTS CO,INDEPENDENT
4,216099309,LOWE'S HOME IMPROVEMENT,LOWE'S HOME IMPROVEMENT


In [27]:
comp_final_df = utils.add_metric_by_dest(master_df, comp_cat_df, 'competition_destination_id', 'competition_name_category', get_dummies=True)
comp_final_df.head()

,origin_id,F1yearincrements_AGE0_CY,F1yearincrements_AGE1_CY,F1yearincrements_AGE2_CY,F1yearincrements_AGE3_CY,F1yearincrements_AGE4_CY,F1yearincrements_AGE5_CY,F1yearincrements_AGE6_CY,F1yearincrements_AGE7_CY,F1yearincrements_AGE8_CY,...,proximity_competition_kilometers_03,destination_competition_id_04,proximity_competition_traveltime_04,proximity_competition_kilometers_04,destination_competition_id_05,proximity_competition_traveltime_05,proximity_competition_kilometers_05,destination_competition_id_06,proximity_competition_traveltime_06,proximity_competition_kilometers_06
0,530530701003,31.0,25.0,30.0,28.0,35.0,35.0,32.0,24.0,37.0,...,25.960217,403218374.0,30.669763,26.656119,948351200.0,33.274677,31.095850,2890986.0,41.531836,34.025060
1,530530714071,27.0,27.0,27.0,25.0,25.0,25.0,25.0,25.0,25.0,...,6.134528,805003878.0,12.530292,8.456627,401706143.0,14.085878,9.183802,460555345.0,12.203891,9.231713
2,530530714072,22.0,23.0,15.0,18.0,22.0,11.0,17.0,29.0,24.0,...,5.910509,460555345.0,9.562275,6.501900,805003878.0,13.746727,9.084260,667567382.0,12.891244,10.126950
3,530530714073,22.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,22.0,...,6.162934,460555345.0,13.333669,7.594656,667567382.0,15.409227,10.195625,667588727.0,17.839612,11.169613
4,530530714112,23.0,22.0,22.0,23.0,22.0,22.0,22.0,22.0,21.0,...,6.268874,718164872.0,10.085421,6.620393,667588727.0,11.028254,7.036569,996721817.0,11.014189,7.203595


In [28]:
[c for c in comp_final_df.columns if c.startswith('destination')]

['destination_id_01',
 'destination_id_02',
 'destination_id_03',
 'destination_id_04',
 'destination_id_05',
 'destination_id_06',
 'destination_competition_id_01',
 'destination_competition_id_02',
 'destination_competition_id_03',
 'destination_competition_id_04',
 'destination_competition_id_05',
 'destination_competition_id_06']

In [69]:
comp_final_df = utils.add_metric_by_dest(closest_comp_df, comp_cat_df, 'dest_id', 'dest_name_category', get_dummies=True)
keep_cols = [c for c in comp_final_df.columns if not c.startswith('destination_id')]
comp_final_df = comp_final_df[keep_cols].copy()
comp_final_df.set_index('origin_id', inplace=True, drop=True)
comp_final_df.head()

,proximity_competition_traveltime_01,proximity_competition_kilometers_01,proximity_competition_traveltime_02,proximity_competition_kilometers_02,proximity_competition_traveltime_03,proximity_competition_kilometers_03,proximity_competition_traveltime_04,proximity_competition_kilometers_04,proximity_competition_traveltime_05,proximity_competition_kilometers_05,...,dest_name_category_06_MANOR_HDW_CONSTRUCTION_SUPL,dest_name_category_06_MC_LENDON_HARDWARE,dest_name_category_06_MCLENDON_HARDWARE_INC,dest_name_category_06_SGS_HARDWARE,dest_name_category_06_SNAPON_TOOLS,dest_name_category_06_TOWN_CENTER_HARDWARE,dest_name_category_06_TRUE_VALUE_HARDWARE,dest_name_category_06_TWEEDY_POPP_HARDWARE,dest_name_category_06_US_TOOL_GROUP,dest_name_category_06_WILCO_FARM_STOREPUYALLUP
origin_id,,,,,,,,,,,,,,,,,,,,,
530530701003,27.324552,24.844065,26.486351,25.466621,30.496295,25.960217,30.669763,26.656119,33.274677,31.095850,...,0,1,0,0,0,0,0,0,0,0
530530714071,5.290436,3.543129,10.072421,5.099409,8.464914,6.134528,12.530292,8.456627,14.085878,9.183802,...,0,0,0,0,0,0,0,0,0,0
530530714072,7.551801,4.354229,7.940114,5.186840,12.333787,5.910509,9.562275,6.501900,13.746727,9.084260,...,0,0,0,0,0,0,0,0,0,0
530530714073,8.712262,4.531160,9.507141,4.721377,11.474098,6.162934,13.333669,7.594656,15.409227,10.195625,...,0,0,0,0,0,0,0,0,0,0
530530714112,2.551648,0.957319,7.529261,5.468934,6.571241,6.268874,10.085421,6.620393,11.028254,7.036569,...,0,0,0,0,0,0,0,0,0,0


In [50]:
brand_df = pd.read_csv(closest_brand, index_col=0)
brand_df.head()

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06
0,530530701003,677129595,43.576384,38.513638,371889957,46.482533,42.526899,427271369,52.472263,44.462895,421027779,55.156955,44.748810,721714069,61.428125,50.456415,216082099,58.905164,54.642485
1,530530714071,371889957,9.405840,6.295963,216082099,13.296163,10.944308,421027779,16.006815,12.366455,721714069,20.814301,15.290477,677129595,29.198017,18.116390,460556608,35.087418,25.237232
2,530530714072,371889957,10.003444,6.125873,216082099,11.536480,8.195827,421027779,18.268181,13.177555,721714069,22.034957,15.623036,677129595,29.203660,20.609116,460556608,34.472733,25.286682
3,530530714073,216082099,12.308627,7.540147,371889957,13.269904,8.468519,421027779,20.223521,13.544703,721714069,23.035677,15.164948,677129595,31.159000,20.976263,460556608,35.244881,24.631002
4,530530714112,216082099,5.292562,3.556479,371889957,12.285190,9.615196,721714069,16.478653,11.395147,421027779,19.885017,15.617594,460556608,28.228815,20.647334,677129595,30.820496,23.049155


In [53]:
trips_df = pd.read_csv(inrix_trips, index_col=0)
trips_df.columns = ['origin_id', 'destination_id', 'trip_count']
trips_df.head()

,origin_id,destination_id,trip_count
0,530330001001,425477834,1
1,530330001001,426576447,1
2,530330001001,439724905,2
3,530330001001,714937939,2
4,530330001001,718542034,1


In [58]:
brand_fnl_df = utils.add_metric_by_origin_dest(brand_df, trips_df, 'trip_count').fillna(0.0)
brand_fnl_df.set_index('origin_id', inplace=True, drop=True)
brand_fnl_df.head()

,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,...,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06,trip_count_01,trip_count_02,trip_count_03,trip_count_04,trip_count_05,trip_count_06
origin_id,,,,,,,,,,,,,,,,,,,,,
530530701003,677129595,43.576384,38.513638,371889957,46.482533,42.526899,427271369,52.472263,44.462895,421027779,...,50.456415,216082099,58.905164,54.642485,0.0,2.0,0.0,0.0,0.0,0.0
530530714071,371889957,9.405840,6.295963,216082099,13.296163,10.944308,421027779,16.006815,12.366455,721714069,...,18.116390,460556608,35.087418,25.237232,2.0,2.0,0.0,0.0,0.0,0.0
530530714072,371889957,10.003444,6.125873,216082099,11.536480,8.195827,421027779,18.268181,13.177555,721714069,...,20.609116,460556608,34.472733,25.286682,3.0,0.0,0.0,0.0,1.0,0.0
530530714073,216082099,12.308627,7.540147,371889957,13.269904,8.468519,421027779,20.223521,13.544703,721714069,...,20.976263,460556608,35.244881,24.631002,1.0,0.0,0.0,0.0,0.0,0.0
530530714112,216082099,5.292562,3.556479,371889957,12.285190,9.615196,721714069,16.478653,11.395147,421027779,...,20.647334,677129595,30.820496,23.049155,7.0,0.0,1.0,0.0,0.0,0.0


In [60]:
demog_df = pd.read_csv(origin_demographics, index_col=0)
demog_df.head()

,F1yearincrements_AGE0_CY,F1yearincrements_AGE1_CY,F1yearincrements_AGE2_CY,F1yearincrements_AGE3_CY,F1yearincrements_AGE4_CY,F1yearincrements_AGE5_CY,F1yearincrements_AGE6_CY,F1yearincrements_AGE7_CY,F1yearincrements_AGE8_CY,F1yearincrements_AGE9_CY,...,yearmovedin_ACSOMV2000,yearmovedin_ACSOMV2010,yearmovedin_ACSOMV2015,yearmovedin_ACSOWNER,yearmovedin_ACSRMV1979,yearmovedin_ACSRMV1980,yearmovedin_ACSRMV1990,yearmovedin_ACSRMV2000,yearmovedin_ACSRMV2010,yearmovedin_ACSRMV2015
ID,,,,,,,,,,,,,,,,,,,,,
530530701003,31.0,25.0,30.0,28.0,35.0,35.0,32.0,24.0,37.0,28.0,...,287.0,101.0,46.0,473.0,0.0,0.0,7.0,49.0,37.0,23.0
530530714071,27.0,27.0,27.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,...,216.0,10.0,0.0,226.0,0.0,0.0,10.0,13.0,110.0,0.0
530530714072,22.0,23.0,15.0,18.0,22.0,11.0,17.0,29.0,24.0,24.0,...,227.0,75.0,0.0,536.0,0.0,0.0,0.0,12.0,14.0,44.0
530530714073,22.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,22.0,23.0,...,100.0,187.0,13.0,528.0,0.0,0.0,0.0,21.0,38.0,21.0
530530714112,23.0,22.0,22.0,23.0,22.0,22.0,22.0,22.0,21.0,21.0,...,121.0,83.0,0.0,273.0,0.0,0.0,31.0,13.0,103.0,0.0


In [70]:
final_df = demog_df.join(comp_final_df).join(brand_fnl_df)
final_df.head()

,F1yearincrements_AGE0_CY,F1yearincrements_AGE1_CY,F1yearincrements_AGE2_CY,F1yearincrements_AGE3_CY,F1yearincrements_AGE4_CY,F1yearincrements_AGE5_CY,F1yearincrements_AGE6_CY,F1yearincrements_AGE7_CY,F1yearincrements_AGE8_CY,F1yearincrements_AGE9_CY,...,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06,trip_count_01,trip_count_02,trip_count_03,trip_count_04,trip_count_05,trip_count_06
ID,,,,,,,,,,,,,,,,,,,,,
530530701003,31.0,25.0,30.0,28.0,35.0,35.0,32.0,24.0,37.0,28.0,...,50.456415,216082099.0,58.905164,54.642485,0.0,2.0,0.0,0.0,0.0,0.0
530530714071,27.0,27.0,27.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,...,18.116390,460556608.0,35.087418,25.237232,2.0,2.0,0.0,0.0,0.0,0.0
530530714072,22.0,23.0,15.0,18.0,22.0,11.0,17.0,29.0,24.0,24.0,...,20.609116,460556608.0,34.472733,25.286682,3.0,0.0,0.0,0.0,1.0,0.0
530530714073,22.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,22.0,23.0,...,20.976263,460556608.0,35.244881,24.631002,1.0,0.0,0.0,0.0,0.0,0.0
530530714112,23.0,22.0,22.0,23.0,22.0,22.0,22.0,22.0,21.0,21.0,...,20.647334,677129595.0,30.820496,23.049155,7.0,0.0,1.0,0.0,0.0,0.0


In [71]:
final_df.to_csv(interim_dir/'training_data.csv')